In [7]:
%load_ext autoreload
%autoreload 2
from call_llm import *
from learn_from_mistake import *
from prompts import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
msg = [{"role": "system", "content": "You are a helpful assistant."},
                                {"role": "user", "content": "What is the capital of France?"}]
response = call_openai_gpt(msg)

print(response)

The capital of France is Paris.


In [18]:
lfm_initial_few_shot = [
    ("int x = 0;", 0),   # NO
    ("char *ptr; free(ptr);", 1)  # YES
]

lfm_training_data = [
    ("int *p; p = malloc(10);", 0),
    ("char *s; strcpy(s, 'Hello');", 1),
    ("int x = 5; x++;", 0),
    ("free(NULL);", 0),
    ("int *arr = malloc(10 * sizeof(int)); arr[10] = 5;", 1)
]

lfm_max_few_shot_size = 5

# Run the updated function
lfm_updated_few_shot = lfm_learn_from_mistakes(
    training_data=lfm_training_data,
    few_shot_set=lfm_initial_few_shot,
    max_few_shot_size=lfm_max_few_shot_size,
    prompt_template=lfm_PROMPT_TEMPLATE  # Pass the template explicitly
)

print("Updated Few-Shot Set:")
for example in lfm_updated_few_shot:
    print(example)


Updated Few-Shot Set:
('int x = 0;', 0)
('char *ptr; free(ptr);', 1)
('int *p; p = malloc(10);', 0)


## Test LFNN

In [25]:
from learn_from_nearest_neighbors import *

# New test example (not seen during training)
new_code_snippet = "memcpy(dest, src, strlen(src));"  # Buffer overflow risk, should be YES

# Run the model using the updated few-shot examples
predicted_label = call_openai_gpt_few_shot(
    system_msg=lfm_SYSTEM_MSG,
    # few_shot_set=[(code, "YES" if lbl == 1 else "NO") for code, lbl in lfm_updated_few_shot],  # Convert labels
    few_shot_set=[],  # Convert labels
    user_question=lfm_PROMPT_TEMPLATE.format(code=new_code_snippet),
    prompt_template=lfm_PROMPT_TEMPLATE
)

print(f"Prediction for new code snippet: {predicted_label}")# Mock encoder model for testing
class MockEncoder:
    def encode(self, demonstration_set, instance):
        return np.random.rand(10)  # Random vector for simplicity

# Instantiate the encoder
encoder_model = MockEncoder()

# Example dataset (function snippets with labels)
training_data = [
    ("int x = 0;", 0),
    ("char *ptr; free(ptr);", 1),
    ("int *p; p = malloc(10);", 0),
    ("char *s; strcpy(s, 'Hello');", 1),
    ("int x = 5; x++;", 0),
    ("free(NULL);", 0),
    ("int *arr = malloc(10 * sizeof(int)); arr[10] = 5;", 1)
]

# Initialize LFNN model with training data
t = 4  # Demonstration set size
lfnn_model = LFNN(training_data, t, encoder_model)

# Query instance (new function to classify)
query_instance = "memcpy(dest, src, strlen(src));"

# Get k nearest neighbors
k = 3
nearest_neighbors = lfnn_model.get_nearest_neighbors(query_instance, k)

# Use nearest neighbors for LLM prediction
prediction = lfnn_predict(
    nearest_neighbors=nearest_neighbors,
    query_instance=query_instance,
    prompt_template=lfm_PROMPT_TEMPLATE
)

# Print results
print("Nearest Neighbors for Few-Shot Learning:")
for example in nearest_neighbors:
    print(example)

print(f"\nGPT-4o Prediction for Query Instance:\n{prediction}")



Prediction for new code snippet: (1) YES: A security vulnerability detected.
Nearest Neighbors for Few-Shot Learning:
('int x = 5; x++;', 0)
('int *arr = malloc(10 * sizeof(int)); arr[10] = 5;', 1)
('char *ptr; free(ptr);', 1)

GPT-4o Prediction for Query Instance:
YES


## Integrate GPT-2 XL Encoder into LFNN

In [26]:
# Initialize LFNN with GPT-2 XL embeddings
lfnn_model = LFNN(training_data, t=4)

# Query a new function and get nearest neighbors
query_instance = "memcpy(dest, src, strlen(src));"
nearest_neighbors = lfnn_model.get_nearest_neighbors(query_instance, k=3)

# Print nearest neighbors
print("Nearest Neighbors:")
for example in nearest_neighbors:
    print(example)

/Users/zhangchi/opt/miniconda3/envs/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Nearest Neighbors:
('char *ptr; free(ptr);', 1)
("char *s; strcpy(s, 'Hello');", 1)
('int *arr = malloc(10 * sizeof(int)); arr[10] = 5;', 1)
